In [14]:
import pandas as pd
from shapely.geometry import Polygon
import geopandas as gpd
from geowrangler import grids
from pathlib import Path


# Create grids
This notebook generates grids for each of the downscaling domains at the specified Bing tile zoom level

### Input parameters

In [3]:
INPUT_PATH = Path("../../data/01-raw/domains")
DEST_PATH = Path("../../data/01-raw/grids")
ZOOM_LEVEL = 15

### Estimate tile size
A reasonable grid for statistical/ML-based downscaling would be around 2-5km size.

In [4]:
CIRCUMFERENCE_EARTH_KM = 40_075

def get_bing_tile_length_m(zoom_level: int) -> float:
    assert isinstance(zoom_level, int) and zoom_level > 0

    num_tiles = 2**zoom_level
    tile_length_km = CIRCUMFERENCE_EARTH_KM / num_tiles
    tile_length_m = tile_length_km * 1000
    return tile_length_m

In [5]:
get_bing_tile_length_m(14)

2445.98388671875

## Generate vector grids

### Read bounds

In [6]:
adm_gdf = gpd.read_file(INPUT_PATH / 'downscaling_domains_fixed.geojson')
adm_gdf 

,city,geometry
0,Dagupan,"POLYGON ((120.00931 15.90723, 120.52218 15.907..."
1,Palayan,"POLYGON ((120.88251 15.29488, 121.33348 15.294..."
2,MetroManila,"POLYGON ((120.86704 14.29347, 121.20453 14.293..."
3,Legazpi,"POLYGON ((123.65317 12.95603, 123.88749 12.956..."
4,Iloilo,"POLYGON ((122.41374 10.59140, 122.69007 10.591..."
5,Mandaue,"POLYGON ((123.84991 10.22738, 124.06435 10.227..."
6,Tacloban,"POLYGON ((124.86370 11.11900, 125.11697 11.119..."
7,Zamboanga,"POLYGON ((121.85371 6.81269, 122.48448 6.81269..."
8,CagayanDeOro,"POLYGON ((124.34436 8.09633, 124.95891 8.09633..."
9,Davao,"POLYGON ((125.05176 6.82890, 125.83580 6.82890..."


In [7]:
# set grid parameters
grid_generator = grids.BingTileGridGenerator(ZOOM_LEVEL, add_xyz_cols=True)

# create grid
grid_gdf = grid_generator.generate_grid_join(gdf=adm_gdf, filter=False)
grid_gdf

,quadkey,geometry
0,132321303012033,"POLYGON ((121.84937 7.58033, 121.84937 7.59122..."
1,132321303012122,"POLYGON ((121.86035 7.58033, 121.86035 7.59122..."
2,132321303012123,"POLYGON ((121.87134 7.58033, 121.87134 7.59122..."
3,132321303012132,"POLYGON ((121.88232 7.58033, 121.88232 7.59122..."
4,132321303012133,"POLYGON ((121.89331 7.58033, 121.89331 7.59122..."
...,...,...
21900,132303013010312,"POLYGON ((120.47607 15.89794, 120.47607 15.908..."
21901,132303013010313,"POLYGON ((120.48706 15.89794, 120.48706 15.908..."
21902,132303013011202,"POLYGON ((120.49805 15.89794, 120.49805 15.908..."
21903,132303013011203,"POLYGON ((120.50903 15.89794, 120.50903 15.908..."


### Overlay with bounds

In [73]:
overlay_gdf = gpd.overlay(grid_gdf, adm_gdf, how='intersection')
overlay_gdf

,quadkey,city,geometry
0,132321303012033,Zamboanga,"POLYGON ((121.86035 7.58033, 121.85371 7.58033..."
1,132321303012122,Zamboanga,"POLYGON ((121.86035 7.59083, 121.87134 7.59083..."
2,132321303012123,Zamboanga,"POLYGON ((121.87134 7.59083, 121.88232 7.59083..."
3,132321303012132,Zamboanga,"POLYGON ((121.88232 7.59083, 121.89331 7.59083..."
4,132321303012133,Zamboanga,"POLYGON ((121.89331 7.59083, 121.90430 7.59083..."
...,...,...,...
21900,132303013010312,Dagupan,"POLYGON ((120.47607 15.90851, 120.48706 15.908..."
21901,132303013010313,Dagupan,"POLYGON ((120.48706 15.90851, 120.49805 15.908..."
21902,132303013011202,Dagupan,"POLYGON ((120.49805 15.90851, 120.50903 15.908..."
21903,132303013011203,Dagupan,"POLYGON ((120.50903 15.90851, 120.52002 15.908..."


### Save to geojson

In [9]:
overlay_gdf.to_file(DEST_PATH / f'downscaling_grid_bing{ZOOM_LEVEL}.geojson', driver='GeoJSON')

/home/jace/miniconda3/envs/climate-downscaling/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


## Generate netcdf grid

In [ ]:
RESOLUTION = 0.02

In [ ]:
ds = xr.open_dataset(DEST_PATH / "data/01-raw/chirts/CHIRTS_Tmax_PH_2007.nc")

In [29]:
cities = adm_gdf['city'].values
for city in

SyntaxError: invalid syntax (248739560.py, line 2)

In [97]:
city = cities[2]
city_grid_gdf = overlay_gdf[overlay_gdf['city']==city].copy()
unique_lons = np.unique(np.concatenate([np.array(poly.exterior.coords.xy[0]) for poly in city_grid_gdf.geometry]))
unique_lats = np.unique(np.concatenate([np.array(poly.exterior.coords.xy[1]) for poly in city_grid_gdf.geometry]))
min_lon,max_lon = min(unique_lons),max(unique_lons)
min_lat,max_lat = min(unique_lats),max(unique_lats)

min_lon, min_lat

(120.8670354518582, 14.293466902449362)

In [ ]:
0.010986,0.01063

In [79]:
city_grid_gdf = overlay_gdf[overlay_gdf['city']=="Mandaue"].copy()
city_grid_gdf

,quadkey,city,geometry
13818,132330002001201,Mandaue,"POLYGON ((123.85986 10.43379, 123.84991 10.433..."
13819,132330002001210,Mandaue,"POLYGON ((123.85986 10.43886, 123.87085 10.438..."
13820,132330002001211,Mandaue,"POLYGON ((123.87085 10.43886, 123.88184 10.438..."
13821,132330002001300,Mandaue,"POLYGON ((123.88184 10.43886, 123.89282 10.438..."
13822,132330002001301,Mandaue,"POLYGON ((123.89282 10.43886, 123.90381 10.438..."
...,...,...,...
14233,132330002033000,Mandaue,"POLYGON ((124.01367 10.22844, 124.02466 10.228..."
14234,132330002033001,Mandaue,"POLYGON ((124.02466 10.22844, 124.03564 10.228..."
14235,132330002033010,Mandaue,"POLYGON ((124.03564 10.22844, 124.04663 10.228..."
14236,132330002033011,Mandaue,"POLYGON ((124.04663 10.22844, 124.05762 10.228..."
